In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import plotly.express as px
from sklearn.linear_model import LinearRegression

In [3]:
# Path del archivo CSV
file_path = "C:/Users/norma/OneDrive/Tecnoquimicas TQ/prueba_tq/data/data_productos.csv"

# Cargar el archivo CSV
df = pd.read_csv(file_path, sep=';')
sku_columns = df.columns.tolist()
df_mensual = df.melt(id_vars=['Fecha'],
                    value_vars=sku_columns,
                    var_name='SKU',
                    value_name='Cantidad')

df_mensual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Fecha     2772 non-null   object
 1   SKU       2772 non-null   object
 2   Cantidad  2772 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 65.1+ KB


In [ ]:
# Cálculo de tendencia para cada SKU
trends = []
for sku, group in df_mensual.groupby('SKU'):
    group['Mes'] = pd.to_datetime(group['Fecha']).dt.to_period('M').apply(lambda x: x.ordinal)  # Convertir tiempo a numérico
    X = group['Mes'].values.reshape(-1, 1)
    y = group['Cantidad'].values.reshape(-1, 1)
    if len(group) > 1:  # Asegurar datos suficientes para regresión
        reg = LinearRegression().fit(X, y)
        trends.append({'SKU': sku, 'Pendiente': reg.coef_[0][0], 'Cantidad_Total': group['Cantidad'].sum()})

df_trends = pd.DataFrame(trends)

In [ ]:
# Filtrar productos con altas ventas y tendencia creciente
percentil_ventas = np.percentile(df_trends['Cantidad_Total'], 75)
productos_filtrados = df_trends[(df_trends['Cantidad_Total'] > percentil_ventas) & (df_trends['Pendiente'] > 0)]

In [5]:
productos_filtrados.head()

,SKU,Pendiente,Cantidad_Total
28,SKU_29,11.094494,916612
